In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext, FunctionTool
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.models.google_llm import Gemini

In [3]:
import APIKey    #Storing my API key in python class "APIKey"
import os

google_API = APIKey.GOOGLESTUDIO_API_KEY    #My google studio API

os.environ["GOOGLE_API_KEY"] = google_API

In [4]:
import yfinance as yf
import pandas_datareader.data as web
import statsmodels.api as sm
import datetime as dt
import pandas_ta as ta

# News Agent

In [5]:
google_news_agent = Agent(
    name="GoogleNewsAgent",
    model="gemini-2.5-flash",
    instruction="""You are a specialized news arrangement agent for a given company. 
    
    For a given company:
    1. Use 'google_search' to find the company's summary.
    2. Read the company's summary and then use 'google_search' to find current related news that would affect the company's stock price.
    3. Generate comment with important points and analysis about the company's stock future performance base on the news.
    """,
    tools = [google_search],
    output_key="google_news_arrangement",
)

# Competitors Analysis Agent

In [6]:
def competitors_compare(target: str) -> dict:
    """Searching for the target company's main competitors and compare their main financial ratio.

    Args:
        taregt: The stock ticker of a company. It should be consisted of uppercase letters,
                e.g., "AAPL" or "NVDA".

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "comparison":
        | Ticker   |   Dividend Yield |   Trailing PE |   TTM PS |   Profit Margin |   PB Ratio |   
        Trailing EPS |   EV/EBITDA |   Current Ratio |   Debt-to-Equity |    ROA |    ROE |   PEG Ratio |\n|:---------|-----------------:|
        --------------:|---------:|----------------:|-----------:|---------------:|------------:|----------------:|-----------------:|-------:|
        -------:|------------:|\n| SHEL     |             3.8  |       15.3694 |   0.8023 |          0.0544 |     2.4892 |           4.9  |
        9.868 |           1.352 |           41.602 | 0.049  | 0.0815 |      1.634  |\n| XOM      |             3.5  |       17.3009 |   1.5554 |
        0.0918 |     1.9265 |           6.88 |       8.626 |           1.137 |           15.672 | 0.0511 | 0.1142 |      2.7756 |\n| CVX      |
        4.42 |       21.6062 |   1.6404 |          0.0677 |     1.6293 |           7.11 |       9.323 |           1.152 |           21.239 |
        0.0405 | 0.0732 |      2.9249 |\n| NFG      |             2.66 |       14.3187 |   3.2274 |          0.2277 |     2.3753 |           5.68 |
        7.132 |           0.444 |           91.548 | 0.0701 | 0.1745 |    N/A      |\n| UNTC     |            15.08 |        4.9303 |   1.3284 |
        0.2334 |     1.3559 |           6.74 |       2.907 |           3.545 |            1.009 | 0.15   | 0.2694 |    N/A      |}
        Error: {"status": "error", "error_message": "Stock ticker not found"}
    """

    metrics = ['dividendYield', 'trailingPE', 'priceToSalesTrailing12Months', 'profitMargins', 'priceToBook', 'trailingEps', 'enterpriseToEbitda', 
    'currentRatio', 'debtToEquity', 'returnOnAssets', 'returnOnEquity', 'trailingPegRatio']
    col_names = ['Ticker', 'Dividend Yield', 'Trailing PE', 'TTM PS', 'Profit Margin', 'PB Ratio', 'Trailing EPS', 'EV/EBITDA', 'Current Ratio',
               'Debt-to-Equity', 'ROA', 'ROE', 'PEG Ratio']
    
    ticker = yf.Ticker(target)
    info = ticker.info
    compare_df = pd.DataFrame(columns = col_names)
    
    industry = yf.Industry(info['industryKey'])
    competitors = list(industry.top_companies.index.values)
    competitors.insert(0, target)
    
    for comp in competitors:
        comp_info = yf.Ticker(comp).info
    
        row_data = [comp]
    
        for metric in metrics:
            value = comp_info.get(metric, 'NaN') 
            row_data.append(value)
    
        compare_df.loc[len(compare_df)] = row_data

    compare_df = compare_df.replace("NaN", np.nan)
    compare_df = compare_df.dropna(axis=1, how='all')
    compare_df = compare_df.fillna("N/A")

    compare_df = compare_df.round(4)
    compare_df_string = compare_df.to_markdown(index=False)

    if compare_df_string is not None:
        return {"status": "success", "comparison": compare_df_string}
    else:
        return {"status": "error", "error_message": f"Stock ticker '{target}' not found"}

In [7]:
competitors_agent = Agent(
    name="CompetitorsAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized agent to compare the target company to its competitors.

    For a given company:
    1. Use 'competitors_compare' to get the target company and its main competitors' financial ratio data.
    2. Check the "status" field in it response for errors.
    3. Aanlyse the financial ratio data, summarize the most important points and provide some insights that might affect the target companies' 
    stock price.
    
    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [competitors_compare],
    output_key="comparing_competitors",
)

# Financial Ratio Agent

In [8]:
def get_financial_ratio(target: str) -> dict:
    """Getting the target company's main financial ratio for the past five years.

    Args:
        taregt: The stock ticker of a company. It should be consisted of uppercase letters,
                e.g., "AAPL" or "NVDA".

    Returns:
        Dictionary with status and financial ratio information.
        Success: {"status": "success", "financial ratio":
        | comp   | fiscalDateEnding    |   GrossMargin |   OperatingMargin |   NetProfitMargin |     ROE |     ROA |    D/E |   CurrentRatio |
        InterestCoverage |   Debt/EBITDA |   AssetTurover |   InventoryTurnover |   ARTurnover |   RevenueGrowth |   NetIncomeGrowth |   EPSGrowth | 
        FCFGrowth |   DividendYield |     P/E |    P/S |    P/B |\n|:-------|:--------------------|--------------:|------------------:|
        ------------------:|--------:|--------:|-------:|---------------:|-------------------:|--------------:|---------------:|--------------------:
        |-------------:|----------------:|------------------:|------------:|------------:|----------------:|--------:|-------:|-------:|\n| SHEL   |
        2020-12-31 00:00:00 |       -0.072  |           -0.1414 |           -0.1201 | -0.1396 | -0.0572 | 1.4212 |         1.2475 |            -5.4055 |
        99.2496 |         0.476  |              9.947  |       8.2188 |         -0.4765 |           -2.3684 |     -0.6946 |     -0.1667 | 
        0.0544 | 28.3387 | 0.7586 | 0.8819 |\n| SHEL   | 2021-12-31 00:00:00 |        0.1371 |            0.0852 |            0.0769 |  0.1169 |
        0.0497 | 1.332  |         1.3477 |             9.272  |        4.0536 |         0.6467 |              8.934  |       7.4833 |          0.4484 |
        -1.9272 |      3.0161 |      0.0318 |          0.0378 |  8.7149 | 0.6478 | 0.9851 |\n| SHEL   | 2022-12-31 00:00:00 |        0.2066 |
        0.1655 |            0.111  |  0.2221 |  0.0955 | 1.3148 |         1.3679 |            13.124  |        2.7057 |         0.8607 |
        9.4856 |       9.6438 |          0.4582 |            1.1048 |      1.1446 |      0.8029 |          0.0348 |  5.3324 | 0.5534 | 1.1078 |\n|
        SHEL   | 2023-12-31 00:00:00 |        0.1491 |            0.0971 |            0.0611 |  0.1037 |  0.0477 | 1.1677 |         1.4048 | 
        8.0499 |        3.6101 |         0.7793 |             10.3545 |       8.6838 |         -0.1697 |           -0.5424 |     -0.2191 | 
        -0.2086 |          0.0376 |  7.8897 | 0.7066 | 1.1988 |\n| SHEL   | 2024-12-31 00:00:00 |        0.1616 |            0.1055 | 
        0.0566 |  0.0903 |  0.0415 | 1.1634 |         1.3461 |             7.1593 |        3.3647 |         0.7335 |             10.1755 |
        9.1038 |         -0.102  |           -0.1687 |     -0.1007 |     -0.0293 |          0.0439 |  8.3533 | 0.7011 | 1.118  |
        Error: {"status": "error", "error_message": "Stock ticker not found"}
    """

    # Get data. Adjust it if doing general
    financial_ratio_df = pd.read_csv("financial_ratio.csv")

    target_ratio_df = financial_ratio_df[financial_ratio_df['comp'] == target].copy()
    target_ratio_df.sort_values(by='fiscalDateEnding', ascending=True)

    cutoff_date = pd.Timestamp.now() - pd.DateOffset(years=5)
    target_ratio_df['fiscalDateEnding'] = pd.to_datetime(target_ratio_df['fiscalDateEnding'])
    target_ratio_df = target_ratio_df[target_ratio_df['fiscalDateEnding'] >= cutoff_date]

    target_ratio_df = target_ratio_df.replace("NA", np.nan)
    target_ratio_df = target_ratio_df.dropna(axis=1, how='all')
    target_ratio_df = target_ratio_df.fillna("N/A")

    target_ratio_df = target_ratio_df.round(4)
    target_ratio_df_string = target_ratio_df.to_markdown(index=False)

    if target_ratio_df_string is not None:
        return {"status": "success", "financial ratio": target_ratio_df_string}
    else:
        return {"status": "error", "error_message": f"Stock ticker '{target}' not found"}

In [9]:
financial_ratio_agent = Agent(
    name="FinancialRatioAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized agent to the analyse the target company's financial ratio for the past 5 years.

    For a given company:
    1. Use 'get_financial_ratio' to get the recent 5 years financial ratio data of the target company.
    2. Check the "status" field in it response for errors.
    3. Aanlyse the financial ratio data, summarize the most important points and provide some insights that might affect the target companies' 
    stock price.
    
    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [get_financial_ratio],
    output_key="financial_ratio_analysis",
)

### Fetching Financial Ratio Data

In [6]:
financial_data = pd.read_csv("financial_data.csv")

financial_ratio = financial_data[['comp', 'fiscalDateEnding']].copy()

financial_ratio['GrossMargin'] = np.where(
    financial_data['totalRevenue'] != 0,
    financial_data['grossProfit'] / financial_data['totalRevenue'],
    np.nan
)

financial_ratio['OperatingMargin'] = np.where(
    financial_data['totalRevenue'] != 0,
    financial_data['operatingIncome'] / financial_data['totalRevenue'],
    np.nan
)

financial_ratio['NetProfitMargin'] = np.where(
    financial_data['totalRevenue'] != 0,
    financial_data['netIncome'] / financial_data['totalRevenue'],
    np.nan
)

financial_ratio['ROE'] = np.where(
    financial_data['totalShareholderEquity'] != 0,
    financial_data['netIncome'] / financial_data['totalShareholderEquity'],
    np.nan
)

financial_ratio['ROA'] = np.where(
    financial_data['totalAssets'] != 0,
    financial_data['netIncome'] / financial_data['totalAssets'],
    np.nan
)

## Cancel ROIC

financial_ratio['D/E'] = np.where(
    financial_data['totalShareholderEquity'] != 0,
    financial_data['totalLiabilities'] / financial_data['totalShareholderEquity'],
    np.nan
)

financial_ratio['CurrentRatio'] = np.where(
    financial_data['totalCurrentLiabilities'] != 0,
    financial_data['totalCurrentAssets'] / financial_data['totalCurrentLiabilities'],
    np.nan
)

financial_ratio['InterestCoverage'] = np.where(
    financial_data['interestExpense'] != 0,
    financial_data['ebit'] / financial_data['interestExpense'],
    np.nan
)

financial_ratio['Debt/EBITDA'] = np.where(
    financial_data['ebitda'] != 0,
    financial_data['totalLiabilities'] / financial_data['ebitda'],
    np.nan
)

financial_ratio['AssetTurover'] = np.where(
    financial_data['totalAssets'] != 0,
    financial_data['totalRevenue'] / financial_data['totalAssets'],
    np.nan
)

financial_ratio['InventoryTurnover'] = np.where(
    financial_data['inventory'] != 0,
    financial_data['costofGoodsAndServicesSold'] / financial_data['inventory'],
    np.nan
)

financial_ratio['ARTurnover'] = np.where(
    financial_data['currentNetReceivables'] != 0,
    financial_data['totalRevenue'] / financial_data['currentNetReceivables'],
    np.nan
)

financial_ratio['ARTurnover'] = np.where(
    financial_data['currentNetReceivables'] != 0,
    financial_data['totalRevenue'] / financial_data['currentNetReceivables'],
    np.nan
)

# Sort the raw data
# financail_data = financial_data.sort_values(by=['comp', 'fiscalDateEnding'], ascending=[True, True])

financial_ratio['RevenueGrowth'] = financial_data.groupby('comp')['totalRevenue'].pct_change()

financial_ratio['NetIncomeGrowth'] = financial_data.groupby('comp')['netIncome'].pct_change()

# EPS Growth
financial_ratio['EPSGrowth'] = financial_data.groupby('comp')['reportedEPS'].pct_change()

# FCF Growth
financial_ratio['FCFGrowth'] = financial_data.groupby('comp')['freeCashFlow'].pct_change()

# Dividend Yield
financial_ratio['DividendYield'] = np.where(
    financial_data['closePrice'] != 0,
    financial_data['dividend'] / financial_data['closePrice'],
    np.nan
)

# P/E
financial_ratio['P/E'] = np.where(
    financial_data['reportedEPS'] != 0,
    financial_data['closePrice'] / financial_data['reportedEPS'],
    np.nan
)

# P/S
financial_ratio['P/S'] = np.where(
    financial_data['totalRevenue'] != 0,
    financial_data['closePrice'] * financial_data['commonStockSharesOutstanding']  / financial_data['totalRevenue'],
    np.nan
)

# P/B
financial_ratio['P/B'] = np.where(
    financial_data['totalShareholderEquity'] != 0,
    financial_data['closePrice'] * financial_data['commonStockSharesOutstanding']  / financial_data['totalShareholderEquity'],
    np.nan
)

# Cancel EV to EBITDA

C:\Users\User\AppData\Local\Temp\ipykernel_24272\738313355.py:93: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  financial_ratio['EPSGrowth'] = financial_data.groupby('comp')['reportedEPS'].pct_change()


In [7]:
financial_ratio.head()

,comp,fiscalDateEnding,GrossMargin,OperatingMargin,NetProfitMargin,ROE,ROA,D/E,CurrentRatio,InterestCoverage,...,InventoryTurnover,ARTurnover,RevenueGrowth,NetIncomeGrowth,EPSGrowth,FCFGrowth,DividendYield,P/E,P/S,P/B
0,SHEL,2005-12-31,0.176405,0.121739,0.082519,0.278375,0.115304,1.337293,1.152159,362.533981,...,12.774171,NaN,NaN,NaN,NaN,NaN,0.185224,9.503864,0.674828,2.276524
1,CVX,2005-12-31,0.289092,0.174036,0.071135,0.224951,0.112045,1.004483,1.372836,NaN,...,34.191216,11.533985,NaN,NaN,NaN,NaN,0.313546,8.680428,0.617484,1.952667
2,XOM,2005-12-31,0.425375,0.163826,0.097470,0.324951,0.173423,0.873752,1.583821,2.176985,...,22.851840,13.487120,NaN,NaN,NaN,NaN,0.158270,10.499065,0.958820,3.196584
3,NFG,2006-09-30,0.434042,0.158783,0.061657,0.095660,0.036690,1.607265,1.515956,4.896433,...,13.182624,9.862977,NaN,NaN,NaN,NaN,0.288308,15.145833,1.396245,2.166263
4,SHEL,2006-12-31,0.170283,0.118170,0.079794,0.240641,0.108137,1.138140,1.197230,135.046595,...,11.395692,8.040069,0.039494,0.005176,0.204019,0.367351,0.034561,9.087291,0.714905,2.155986


In [9]:
financial_ratio.to_csv("financial_ratio.csv", index=False, na_rep='NA')

# Fair Value Agent

In [10]:
def fair_value_calcualtion(target: str) -> dict:
    """Calculate the target company's fair value using a DCF model to determine if it is over/undervalued.

    Args:
        target: The stock ticker of a company (e.g., "AAPL", "NVDA"). 
               Must be comprised of uppercase letters.

    Returns:
        dict: A structured dictionary containing valuation metrics and status.
        
        Success Response Example:
            {
                "status": "success",
                "ticker": "SHEL",
                "currency": "USD",
                "metrics": {
                    "current_price": 72.81,
                    "fair_value": 270.35,
                    "wacc": 0.0834,
                    "cost_of_equity": 0.095,
                    "cost_of_debt": 0.042,
                    "projected_fcf_per_share": 22.55,
                    "growth_assumption": 0.03
                },
                "conclusion": "Undervalued"
            }

        Error Response Example:
            {
                "status": "error", 
                "error_message": "Stock ticker not found"
            }
    """
    
    ticker = yf.Ticker(target)

    lookback_years = 5
    try:
        estimates = ticker.get_growth_estimates()
        if estimates is not None and not estimates.empty:
            growth_rate_projection = estimates.loc['+1y', 'stockTrend']
        else:
            raise ValueError("Empty estimates data")
    except (KeyError, AttributeError, TypeError, ValueError) as e:
        growth_rate_projection = 0.00
    projection_years = 5
    terminal_growth_rate = 0.00
    err_tolerence = 0.05

    end_date = dt.datetime.now()
    start_date = end_date - dt.timedelta(days=lookback_years*365)

    # Get data. Adjust it if doing general
    try:
        financial_data_df = pd.read_csv("financial_data.csv")
    except FileNotFoundError:
        return {"status": "error", "error_message": "Financial data file not found."}
        
    target_df = financial_data_df[financial_data_df['comp'] == target].copy()
    if target_df.empty:
        return {"status": "error", "error_message": f"Ticker {target} not found in financial data."}
    target_df = target_df.sort_values(by='fiscalDateEnding', ascending=False)
    
    try:
        ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
        ff_data = ff_data / 100
        ff_data.index = ff_data.index.to_timestamp()
    except Exception:
        return {"status": "error", "error_message": f"Failed to fetch Fama-French data: {str(e)}"}

    stock_hist = ticker.history(start=start_date-pd.DateOffset(months=2), end=end_date, interval='1mo')
    if stock_hist.empty:
        return {"status": "error", "error_message": "No price history found from Yahoo Finance."}

    stock_returns = stock_hist['Close'].pct_change().dropna()
    stock_returns.index = stock_returns.index.to_period('M')
    ff_data.index = ff_data.index.to_period('M')

    data = pd.merge(stock_returns, ff_data, left_index=True, right_index=True)
    data.columns = ['Stock_Return', 'Mkt-RF', 'SMB', 'HML', 'RF']
    data['Excess_Return'] = data['Stock_Return'] - data['RF']
    if data.empty:
        return {"status": "error", "error_message": "Not enough overlapping data for regression."}
    
    X = sm.add_constant(data[['Mkt-RF', 'SMB', 'HML']])
    model = sm.OLS(data['Excess_Return'], X).fit()
    
    coe = (data['RF'].mean()
        + (model.params['Mkt-RF'] * data['Mkt-RF'].mean()) 
        + (model.params['SMB'] * data['SMB'].mean()) 
        + (model.params['HML'] * data['HML'].mean())) * 12

    net_income = target_df['netIncome']
    fcf = target_df['freeCashFlow']
        
    ratios = (fcf / net_income).replace([np.inf, -np.inf], np.nan).dropna()
    if ratios.empty:
        return {"status": "error", "error_message": "Missing net income or free cash flow data."}
    avg_ratio = ratios.mean()
        
    forward_eps = ticker.info.get('forwardEps') or ticker.info.get('trailingEps')
    shares = ticker.info.get('sharesOutstanding')
    current_price = ticker.info.get('currentPrice')
    currency = ticker.info.get('currency', 'USD')
    if not all([forward_eps, shares, current_price]):
        return {"status": "error", "error_message": "Missing key metrics (EPS, Shares, or Price) from Yahoo Finance."}
        
    forward_fcf_per_share = forward_eps * avg_ratio
    
    int_exp = target_df.iloc[0]['interestExpense']
    debt = target_df.iloc[0]['shortLongTermDebtTotal']
    tax_rate = target_df['taxRate'].mean()
    if debt > 0:
        cod = (int_exp / debt) * (1 - tax_rate)
    else:
        cod = 0.00
    
    market_cap = shares * current_price
    total_value = market_cap + debt
    wacc = ((market_cap / total_value) * coe) + ((debt / total_value) * cod)
    # Safety clamp: If WACC is crazy (negative or tiny), floor it
    if wacc < 0.05: wacc = 0.05
    if wacc <= terminal_growth_rate:
        return {"status": "error", "error_message": "WACC is lower than terminal growth rate (Math Infinite)."}
    
    future_fcf = [(forward_fcf_per_share * shares) * ((1 + growth_rate_projection) ** i) for i in range(1, projection_years + 1)]

    term_val = (future_fcf[-1] * (1 + terminal_growth_rate)) / (wacc - terminal_growth_rate)
    disc_fcfs = sum([f / ((1 + wacc) ** (i + 1)) for i, f in enumerate(future_fcf)])
    disc_tv = term_val / ((1 + wacc) ** projection_years)
    
    intrinsic_val = (disc_fcfs + disc_tv) / shares
    lower_end = intrinsic_val * (1 - err_tolerence)
    higher_end = intrinsic_val * (1 + err_tolerence)
    if current_price > higher_end:
        status = "Overvalued"
    elif current_price < lower_end:
        status = "Undervalued"
    else:
        status = "Fairvalued"

    result_data = {
        "status": "success",
        "ticker": target,
        "currency": currency,
        "metrics": {
            "current_price": round(current_price, 2),
            "fair_value": round(intrinsic_val, 2),
            "wacc": round(wacc, 4),
            "cost_of_equity": round(coe, 4),
            "cost_of_debt": round(cod, 4),
            "projected_fcf_per_share": round(forward_fcf_per_share, 2),
            "growth_assumption": growth_rate_projection,
        },
        "conclusion": status,
    }
    
    return result_data

In [11]:
fair_value_agent = Agent(
    name="FairValueAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized agent to the analyse the target company's fair value comparing to its current value.

    For a given company:
    1. Use 'fair_value_calcualtion' to get the price information of the target company.
    2. Check the "status" field in it response for errors.
    3. Aanlyse the given price information, summarize the most important points and provide some insights about the target companies' future stock price.
    
    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [fair_value_calcualtion],
    output_key="fair_value_analysis",
)

### Testing Function Result

In [28]:
test = fair_value_calcualtion('SHEL')
print(test)

C:\Users\User\AppData\Local\Temp\ipykernel_25776\1681592820.py:65: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
C:\Users\User\AppData\Local\Temp\ipykernel_25776\1681592820.py:65: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
C:\Users\User\AppData\Local\Temp\ipykernel_25776\1681592820.py:76: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_returns.index = stock_returns.index.to_period('M')


{'status': 'success', 'ticker': 'SHEL', 'currency': 'USD', 'metrics': {'current_price': 72.81, 'fair_value': np.float64(71.57), 'wacc': np.float64(0.104), 'cost_of_equity': np.float64(0.1276), 'cost_of_debt': np.float64(0.0407), 'projected_fcf_per_share': np.float64(7.27), 'growth_assumption': np.float64(0.0058)}, 'conclusion': 'Fairvalued'}


# Momentem & Sentiment Agent

In [12]:
def get_technical_analysis(target: str) -> dict:

    """Getting the target company's main technical indicators for the past ten years.

    Args:
        target: The stock ticker of a company (e.g., "AAPL", "NVDA"). 
               Must be comprised of uppercase letters.

    Returns:
        dict: Dictionary with status and the stock's technical indicators data.

         Success Response Example:
            {'status': 'success',
             'technical indicator': '| Date                |   Close |   Volume |     RSI |    MACD |     ADX | ADX_Dir   |     MFI |   Volume_Change | 
             MA_10 |   MA_20 |   MA_50 |   MA_200 |   dist_to_S1 |   dist_to_R1 |\n|:--------------------|--------:|---------:|--------:|--------:|
             --------:|:----------|--------:|----------------:|--------:|--------:|--------:|---------:|-------------:|-------------:|\n|
             2025-10-03 00:00:00 | 72.5196 |  4230749 | 56.4217 |  0.0757 | 11.7615 | Bull      | 57.245  |         14.0219 | 71.5577 | 71.3046 |
             71.3795 |  66.8196 |       0.0229 |       0.014  |\n| 2025-10-06 00:00:00 | 73.5696 |  4018859 | 61.3016 |  0.1812 | 12.3895 | Bull      |
             55.2783 |         -5.0083 | 71.8767 | 71.4314 | 71.4388 |  66.8891 |       0.0217 |       0.0094 |\n| 2025-10-07 00:00:00 | 74.2035 |
             4000764 | 63.9281 |  0.2775 | 13.4326 | Bull      | 64.9389 |         -0.4503 | 72.2105 | 71.5795 | 71.5064 |  66.963  |       0.0181 |
             0.003  |\n| 2025-10-08 00:00:00 | 73.421  |  3343414 | 58.6377 |  0.272  | 14.1315 | Bull      | 65.3229 |        -16.4306 | 72.4086 |
             71.6523 | 71.5407 |  67.0391 |      -0.0024 |      -0.016  |\n| 2025-10-09 00:00:00 | 72.8365 |  3716811 | 54.9784 |  0.2149 | 14.3347 |
             Bull      | 65.2773 |         11.1681 | 72.472  | 71.7271 | 71.5905 |  67.1115 |      -0.0023 |      -0.0155 |\n| 2025-10-10 00:00:00 |
             70.7464 |  4356906 | 44.3251 |  0.032  | 13.8596 | Bear      | 58.2085 |         17.2216 | 72.2759 | 71.7311 | 71.589  |  67.1742 | 
             -0.0213 |      -0.0444 |\n| 2025-10-13 00:00:00 | 71.4101 |  2649090 | 47.785  | -0.0445 | 13.4184 | Bear      | 56.7345 |        -39.1979 |
             72.2402 | 71.7519 | 71.6048 |  67.2376 |       0.0158 |      -0.0035 |\n| 2025-10-14 00:00:00 | 70.9743 |  2318093 | 45.7733 | -0.1204 |
             13.3349 | Bear      | 51.1241 |        -12.4947 | 72.2521 | 71.7182 | 71.6149 |  67.2976 |      -0.0015 |      -0.0095 |\n|
             2025-10-15 00:00:00 | 71.2021 |  2473845 | 47.0286 | -0.1494 | 12.9821 | Bear      | 49.4382 |          6.719  | 72.1976 | 71.7415 |
             71.6187 |  67.359  |       0.0089 |      -0.0025 |\n| 2025-10-16 00:00:00 | 70.6969 |  3888248 | 44.5649 | -0.1938 | 13.0727 | Bear      |
             41.4928 |         57.1743 | 72.158  | 71.738  | 71.6156 |  67.4164 |      -0.0034 |      -0.0122 |\n| 2025-10-17 00:00:00 | 71.9054 |
             3655692 | 51.1568 | -0.1358 | 12.9402 | Bear      | 49.2525 |         -5.981  | 72.0966 | 71.8271 | 71.6537 |  67.4793 |       0.0234 |
             0.0062 |\n| 2025-10-20 00:00:00 | 72.1332 |  4213849 | 52.3082 | -0.0788 | 12.3489 | Bear      | 57.6872 |         15.2682 | 71.953  |
             71.9148 | 71.6909 |  67.5392 |       0.011  |      -0.0011 |\n| 2025-10-21 00:00:00 | 71.9846 |  3310152 | 51.4563 | -0.0495 | 11.9107 |
             Bear      | 50.7719 |        -21.4459 | 71.7311 | 71.9708 | 71.7298 |  67.5963 |       0.0009 |      -0.0055 |\n| 2025-10-22 00:00:00 |
             73.837  |  4081699 | 60.1663 |  0.089  | 11.8746 | Bull      | 58.5266 |         23.3085 | 71.7727 | 72.0906 | 71.7892 |  67.6557 | 
             0.0288 |       0.0197 |\n| 2025-10-23 00:00:00 | 75.0059 |  5457516 | 64.4954 |  0.2446 | 12.9929 | Bull      | 59.6808 |         33.707  |
             71.9896 | 72.2308 | 71.8623 |  67.7199 |       0.0252 |       0.0107 |\n| 2025-10-24 00:00:00 | 75.1247 |  5347958 | 64.913  |  0.3354 | 
             14.0492 | Bull      | 60.8093 |         -2.0075 | 72.4274 | 72.3517 | 71.9399 |  67.7791 |       0.0049 |      -0.0036 |\n|
             2025-10-27 00:00:00 | 74.7979 |  4089329 | 62.728  |  0.3515 | 14.6189 | Bull      | 52.9789 |        -23.5348 | 72.7662 | 72.5032 |
             72.0124 |  67.8424 |      -0.0004 |      -0.0092 |\n| 2025-10-28 00:00:00 | 74.362  |  4083949 | 59.8358 |  0.3122 | 14.8574 | Bull      |
             52.191  |         -0.1316 | 73.105  | 72.6785 | 72.0829 |  67.9025 |      -0.002  |      -0.0098 |\n| 2025-10-29 00:00:00 | 74.8375 | 
             6587149 | 61.8996 |  0.2977 | 15.658  | Bull      | 61.3383 |         61.2936 | 73.4685 | 72.8331 | 72.1641 |  67.9648 |       0.0092 | 
             0.002  |\n| 2025-10-30 00:00:00 | 74.0252 |  6170798 | 56.5534 |  0.2166 | 15.7247 | Bull      | 59.1946 |         -6.3207 | 73.8014 |
             72.9797 | 72.2164 |  68.0186 |      -0.0051 |      -0.0192 |\n| 2025-10-31 00:00:00 | 74.2134 |  5353128 | 57.47   |  0.1614 | 15.4626 |
             Bull      | 52.2529 |        -13.2506 | 74.0322 | 73.0644 | 72.2646 |  68.0711 |       0.0105 |      -0.0125 |\n| 2025-11-03 00:00:00 |
             73.6884 |  3788745 | 54.0448 |  0.0795 | 15.1507 | Bull      | 50.9481 |        -29.2237 | 74.1877 | 73.0703 | 72.2858 |  68.1205 |   
             -0.0004 |      -0.0129 |\n| 2025-11-04 00:00:00 | 73.0446 |  3762097 | 50.1011 | -0.0227 | 14.2191 | Bear      | 46.1365 |         -0.7033 |
             74.2937 | 73.0124 | 72.2842 |  68.1655 |      -0.0056 |      -0.0136 |\n| 2025-11-05 00:00:00 | 73.9064 |  3703155 | 54.8501 | -0.0361 | 
             13.7625 | Bull      | 51.9742 |         -1.5667 | 74.3006 | 73.0366 | 72.303  |  68.2145 |       0.0174 |       0.0058 |\n| 
             2025-11-06 00:00:00 | 73.52   |  4146631 | 52.4407 | -0.0727 | 13.2884 | Bull      | 45.9815 |         11.9756 | 74.152  | 73.0708 |
             72.3131 |  68.2655 |       0.0003 |      -0.0106 |\n| 2025-11-07 00:00:00 | 74.8771 |  3630586 | 59.2178 | -0.0105 | 13.4867 | Bull      |
             45.595  |        -12.4449 | 74.1272 | 73.2773 | 72.346  |  68.3199 |       0.0218 |       0.0119 |\n| 2025-11-10 00:00:00 | 75.838  | 
             4293402 | 63.2146 |  0.0851 | 14.4124 | Bull      | 51.5319 |         18.2564 | 74.2313 | 73.4987 | 72.3991 |  68.3852 |       0.0241 | 
             0.0066 |\n| 2025-11-11 00:00:00 | 76.2837 |  4132390 | 64.9317 |  0.1636 | 15.7597 | Bull      | 51.6895 |         -3.7502 | 74.4234 | 
             73.7642 | 72.4629 |  68.4539 |       0.0178 |      -0.0009 |\n| 2025-11-12 00:00:00 | 75.6894 |  3912880 | 60.853  |  0.1604 | 16.5009 |
             Bull      | 44.2176 |         -5.3119 | 74.5086 | 73.9886 | 72.5356 |  68.5224 |      -0.0042 |      -0.0127 |\n| 2025-11-13 00:00:00 |
             75.204  |  4360533 | 57.6672 |  0.1129 | 16.4426 | Bull      | 36.232  |         11.4405 | 74.6265 | 74.2139 | 72.6071 |  68.5874 |   
             -0.0045 |      -0.0099 |\n| 2025-11-14 00:00:00 | 75.76   |  4219442 | 60.2353 |  0.1062 | 16.3034 | Bull      | 42.9952 |         -3.2356 |
             74.7812 | 74.4067 | 72.7038 |  68.6463 |       0.0135 |      -0.0016 |\n| 2025-11-17 00:00:00 | 75.4    |  4304263 | 57.7907 |  0.0668 | 
             16.2318 | Bull      | 42.804  |          2.0102 | 74.9523 | 74.57   | 72.7912 |  68.7072 |       0.0036 |      -0.0098 |\n|
             2025-11-18 00:00:00 | 75.31   |  5506234 | 57.1661 |  0.026  | 15.0876 | Bear      | 32.7814 |         27.9251 | 75.1789 | 74.7363 |
             72.8725 |  68.7709 |       0.0041 |      -0.0075 |\n| 2025-11-19 00:00:00 | 73.67   |  5737472 | 47.1627 | -0.1125 | 14.6822 | Bear      |
             33.0551 |          4.1996 | 75.1552 | 74.7279 | 72.9066 |  68.8194 |      -0.0095 |      -0.03   |\n| 2025-11-20 00:00:00 | 73.21   |  
             4992140 | 44.795  | -0.2294 | 13.7558 | Bull      | 41.4141 |        -12.9906 | 75.1242 | 74.6381 | 72.944  |  68.8661 |      -0.0022 |
             -0.0089 |\n| 2025-11-21 00:00:00 | 73.27   |  4574428 | 45.1816 | -0.2929 | 13.0964 | Bear      | 40.9162 |         -8.3674 | 74.9635 |
             74.5454 | 72.9961 |  68.9148 |       0.0072 |      -0.0119 |\n| 2025-11-24 00:00:00 | 73.23   |  3535462 | 44.9556 | -0.3254 | 12.6895 |
             Bear      | 41.0664 |        -22.7125 | 74.7027 | 74.467  | 73.0408 |  68.9647 |       0.0055 |      -0.0048 |\n| 2025-11-25 00:00:00 | 
             72.8    |  5421724 | 42.4946 | -0.3608 | 12.2195 | Bear      | 34.1411 |         53.3526 | 74.3543 | 74.3889 | 73.0638 |  69.0112 |    
             0.0017 |      -0.0105 |\n| 2025-11-26 00:00:00 | 73.41   |  3504885 | 46.9327 | -0.3291 | 11.5403 | Bear      | 40.058  |        -35.3548 | 
             74.1264 | 74.3175 | 73.1173 |  69.0548 |       0.0117 |       0.0023 |\n| 2025-11-28 00:00:00 | 73.77   |  1682586 | 49.4139 | -0.2721 |
             10.9572 | Bull      | 38.1011 |        -51.9931 | 73.983  | 74.3047 | 73.1774 |  69.1032 |       0.0087 |       0.0012 |\n| 
             2025-12-01 00:00:00 | 74.31   |  3304283 | 52.9664 | -0.1902 | 11.1678 | Bull      | 37.0061 |         96.3812 | 73.838  | 74.3096 | 
             73.2611 |  69.1513 |       0.0123 |       0.0026 |\n| 2025-12-02 00:00:00 | 73.96   |  3716093 | 50.4914 | -0.1533 | 10.7392 | Bull      |
             30.117  |         12.4629 | 73.694  | 74.3232 | 73.3327 |  69.1954 |      -0.0006 |      -0.0108 |\n| 2025-12-03 00:00:00 | 75.12   | 
             3818739 | 57.5683 | -0.0498 | 11.0443 | Bull      | 36.7641 |          2.7622 | 73.675  | 74.4269 | 73.4178 |  69.242  |       0.0205 | 
             0.0105 |\n| 2025-12-04 00:00:00 | 74.5    |  3698887 | 53.1918 | -0.0235 | 11.0872 | Bull      | 37.2149 |         -3.1385 | 73.758  | 
             74.4566 | 73.479  |  69.2865 |      -0.0029 |      -0.0111 |\n| 2025-12-05 00:00:00 | 73.01   |  4610019 | 44.4468 | -0.1025 | 10.7495 |
             Bear      | 29.6689 |         24.6326 | 73.738  | 74.4311 | 73.4952 |  69.3198 |      -0.0154 |      -0.0275 |\n| 2025-12-08 00:00:00 | 
             72.7    |  3781087 | 42.8677 | -0.1677 | 10.6991 | Bear      | 30.0086 |        -17.9811 | 73.681  | 74.3223 | 73.495  |  69.3596 |    
             0.0008 |      -0.0141 |\n| 2025-12-09 00:00:00 | 72.55   |  2714033 | 42.0885 | -0.2103 | 10.7814 | Bear      | 31.6001 |        -28.2208 |
             73.613  | 74.1579 | 73.5107 |  69.3978 |       0.0024 |      -0.0106 |\n| 2025-12-10 00:00:00 | 72.88   |  3396839 | 44.4795 | -0.2054 | 
             11.0642 | Bear      | 33.0254 |         25.1584 | 73.621  | 73.9877 | 73.5512 |  69.4377 |       0.0087 |      -0.0034 |\n| 
             2025-12-11 00:00:00 | 72.86   |  3556691 | 44.36   | -0.1929 | 11.1816 | Bear      | 31.0972 |          4.7059 | 73.566  | 73.8462 | 
             73.5734 |  69.4749 |       0.0059 |      -0.0045 |\n| 2025-12-12 00:00:00 | 72.33   |  3693241 | 41.2001 | -0.2083 | 11.9745 | Bear      |
             31.6659 |          3.8392 | 73.422  | 73.7025 | 73.5982 |  69.5102 |      -0.0039 |      -0.0112 |\n| 2025-12-15 00:00:00 | 72.23   | 
             3934013 | 40.6123 | -0.2125 | 12.3533 | Bear      | 39.0708 |          6.5193 | 73.214  | 73.526  | 73.5924 |  69.544  |       0.004  |
             -0.0057 |\n| 2025-12-16 00:00:00 | 70.46   |  5381812 | 31.9289 | -0.3155 | 14.0035 | Bear      | 39.2361 |         36.8021 | 72.864  | 
             73.279  | 73.5302 |  69.5739 |      -0.0191 |      -0.0325 |\n| 2025-12-17 00:00:00 | 71.55   |  4871136 | 40.3825 | -0.2919 | 14.9021 |
             Bear      | 40.6898 |         -9.4889 | 72.507  | 73.091  | 73.4771 |  69.6104 |       0.0223 |       0.006  |\n| 2025-12-18 00:00:00 | 
             71.31   |  5264853 | 39.2273 | -0.2743 | 15.8223 | Bear      | 35.1102 |          8.0827 | 72.188  | 72.973  | 73.4349 |  69.6463 |    
             0.0017 |      -0.0077 |\n| 2025-12-19 00:00:00 | 72.02   |  5581282 | 44.3032 | -0.2002 | 15.6237 | Bear      | 37.5207 |          6.0102 |
             72.089  | 72.9135 | 73.4186 |  69.6861 |       0.0137 |       0.0062 |\n| 2025-12-22 00:00:00 | 72.58   |  3543242 | 47.9928 | -0.1037 | 
             14.7772 | Bear      | 43.8177 |        -36.5156 | 72.077  | 72.879  | 73.4552 |  69.7249 |       0.0115 |       0.0026 |\n| 
             2025-12-23 00:00:00 | 73.07   |  2421211 | 51.0484 | -0.0023 | 13.9912 | Bear      | 42.3137 |        -31.6668 | 72.129  | 72.871  |
             73.4884 |  69.7662 |       0.0093 |       0.0015 |\n| 2025-12-24 00:00:00 | 72.84   |  1021045 | 49.576  |  0.0507 | 13.2613 | Bear      |
             44.5112 |        -57.8292 | 72.125  | 72.873  | 73.5257 |  69.8063 |       0.0001 |      -0.0051 |\n| 2025-12-26 00:00:00 | 72.81   | 
             1580789 | 49.376  |  0.0834 | 12.8186 | Bear      | 47.2236 |         54.8207 | 72.12   | 72.843  | 73.5579 |  69.8434 |       0.0013 |  
             -0.0024 |\n| 2025-12-29 00:00:00 | 73.12   |  2627256 | 51.5516 |  0.1231 | 11.9285 | Bull      | 53.7032 |         66.199  | 72.199  | 
             72.8105 | 73.6064 |  69.8803 |       0.0078 |       0.0012 |'}
         Error Response Example:
            {"status": "error", "error_message": "Stock ticker not found"}
    """
    
    ticker = yf.Ticker(target)
    lookback_years = 10
    analysis_days = 60
    end_date = dt.datetime.now()
    start_date = end_date - dt.timedelta(days=lookback_years*365)
    
    stock_hist = ticker.history(start=start_date, end=end_date, interval='1d', auto_adjust=True)
    if stock_hist.empty:
        return {"status": "error", "error_message": "No price history found from Yahoo Finance."}
    stock_hist.index = stock_hist.index.tz_localize(None)
    if stock_hist.index[-1].date() == dt.date.today():
        stock_hist = stock_hist.iloc[:-1]
    stock_hist = stock_hist.sort_index(ascending=True)
    
    technical_indicator = stock_hist[['Close', 'Volume']].copy()

    # =========================
    # Momentum Indicators
    # =========================
    technical_indicator["RSI"] = ta.rsi(stock_hist["Close"], length=14)

    technical_indicator['MACD'] = ta.macd(stock_hist["Close"])['MACDh_12_26_9']

    adx = ta.adx(
        high=stock_hist["High"],
        low=stock_hist["Low"],
        close=stock_hist["Close"],
        length=14
    )
    technical_indicator['ADX'] = adx['ADX_14']
    technical_indicator['ADX_Dir'] = np.where(adx['DMP_14'] > adx['DMN_14'], "Bull", "Bear")

    # =========================
    # Sentiment Indicators
    # =========================
    technical_indicator["MFI"] = ta.mfi(
        high=stock_hist["High"],
        low=stock_hist["Low"],
        close=stock_hist["Close"],
        volume=stock_hist["Volume"],
        length=14
    )

    technical_indicator["Volume_Change"] = stock_hist["Volume"].pct_change() * 100

    technical_indicator['MA_10'] = ta.sma(stock_hist['Close'], length = 10)
    technical_indicator['MA_20'] = ta.sma(stock_hist['Close'], length = 20)
    technical_indicator['MA_50'] = ta.sma(stock_hist['Close'], length = 50)
    technical_indicator['MA_200'] = ta.sma(stock_hist['Close'], length = 200)

    sr = ta.pivots(open_=stock_hist['Open'], high=stock_hist['High'], low=stock_hist['Low'], close=stock_hist['Close'], method='traditional', anchor='D')
    technical_indicator['dist_to_S1'] = (stock_hist['Close'] - sr['PIVOTS_TRAD_D_S1']) / stock_hist['Close']
    technical_indicator['dist_to_R1'] = (stock_hist['Close'] - sr['PIVOTS_TRAD_D_R1']) / stock_hist['Close']

    # =========================
    # Final Report
    # =========================
    
    technical_indicator = technical_indicator.replace("nan", np.nan)
    technical_indicator = technical_indicator.dropna(axis=0, how='any')
    technical_indicator = technical_indicator.tail(analysis_days)

    technical_indicator = technical_indicator.round(4)
    technical_indicator_string = technical_indicator.to_markdown(index=True)

    if technical_indicator_string is not None:
        return {"status": "success", "technical indicator": technical_indicator_string}
    else:
        return {"status": "error", "error_message": f"Stock ticker '{target}' not found"}

In [13]:
technical_analysis_agent = Agent(
    name="TechnicalAnalysisAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized agent to the analyse the target company's stock technical indicators.

    For a given company:
    1. Use 'get_technical_analysis' to get the stock technical indicators of the target company.
    2. Check the "status" field in it response for errors.
    3. Aanlyse the given data, summarize the most important points and provide some insights about the target companies' future stock price.
    
    If any tool returns status "error", explain the issue to the user clearly.""",
    tools = [get_technical_analysis],
    output_key="technical_analysis",
)

### Testing Function Result

In [23]:
test = get_technical_analysis('SHEL')
test

{'status': 'success',
 'technical indicator': '| Date                |   Close |   Volume |     RSI |    MACD |     ADX | ADX_Dir   |     MFI |   Volume_Change |   MA_10 |   MA_20 |   MA_50 |   MA_200 |   dist_to_S1 |   dist_to_R1 |\n|:--------------------|--------:|---------:|--------:|--------:|--------:|:----------|--------:|----------------:|--------:|--------:|--------:|---------:|-------------:|-------------:|\n| 2025-10-03 00:00:00 | 72.5196 |  4230749 | 56.4217 |  0.0757 | 11.7615 | Bull      | 57.245  |         14.0219 | 71.5577 | 71.3046 | 71.3795 |  66.8196 |       0.0229 |       0.014  |\n| 2025-10-06 00:00:00 | 73.5696 |  4018859 | 61.3016 |  0.1812 | 12.3895 | Bull      | 55.2783 |         -5.0083 | 71.8767 | 71.4314 | 71.4388 |  66.8891 |       0.0217 |       0.0094 |\n| 2025-10-07 00:00:00 | 74.2035 |  4000764 | 63.9281 |  0.2775 | 13.4326 | Bull      | 64.9389 |         -0.4503 | 72.2105 | 71.5795 | 71.5064 |  66.963  |       0.0181 |       0.003  |\n| 2025-10-08 00:00

# Aggregator Agent

In [14]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash",
    instruction="""
    Combine these analyses into a single investment memo:

    **News:**
    {google_news_arrangement}
    
    **Comparison:**
    {comparing_competitors}

    **Financial Ratio:**
    {financial_ratio_analysis}

    **Fair Value:**
    {fair_value_analysis}

    **Technical Indicators:**
    {technical_analysis}
    
    Your memo should highlight the target stock price, recommendations, advantages, disadvantages, and the risk from these analyses.
    The final summary should be around 300 words.
    
    """,
    output_key="investment_memo", # This will be the final output of the entire system.
)

# Output

In [15]:
parallel_analysis_team = ParallelAgent(
    name="ParallelAnalysisTeam",
    sub_agents=[google_news_agent, competitors_agent, financial_ratio_agent, fair_value_agent, technical_analysis_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="AnalysisSystem",
    sub_agents=[parallel_analysis_team, aggregator_agent],
)

In [17]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("SHEL", verbose = True)


 ### Created new session: debug_session_id

User > SHEL


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001C4C8BC5010>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001C4C8BF7D90>


FinancialRatioAgent > [Calling tool: get_financial_ratio({'target': 'SHEL'})]
FairValueAgent > [Calling tool: fair_value_calcualtion({'target': 'SHEL'})]
CompetitorsAgent > [Calling tool: competitors_compare({'target': 'SHEL'})]


C:\Users\User\AppData\Local\Temp\ipykernel_8168\1681592820.py:65: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
C:\Users\User\AppData\Local\Temp\ipykernel_8168\1681592820.py:65: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start_date, end_date)[0]
C:\Users\User\AppData\Local\Temp\ipykernel_8168\1681592820.py:76: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  stock_returns.index = stock_returns.index.to_period('M')
C:\Users\User\AppData\Local\Temp\ipykernel_8168\337580190.py:46

FinancialRatioAgent > [Tool result: {'status': 'success', 'financial ratio': '| comp   | fiscalDateEnding    |   GrossMargin |   Operati...]
FairValueAgent > [Tool result: {'status': 'success', 'ticker': 'SHEL', 'currency': 'USD', 'metrics': {'current_price': 73.66, 'fair...]
TechnicalAnalysisAgent > [Calling tool: get_technical_analysis({'target': 'SHEL'})]
CompetitorsAgent > [Tool result: {'status': 'success', 'comparison': '| Ticker   |   Dividend Yield | Trailing PE   |   TTM PS |   Pr...]
TechnicalAnalysisAgent > [Tool result: {'status': 'success', 'technical indicator': '| Date                |   Close |   Volume |     RSI |...]
FairValueAgent > The current price of SHEL is 73.66 USD. The calculated fair value is 71.43 USD. The company is currently considered fairly valued.
CompetitorsAgent > The provided data contains financial ratios for SHEL and its competitors. Here's a summary of the key points and potential impacts on stock prices:

**Profitability:**
*   **Profit Margin:**